# Catching Criminals with Math
### Identification of fraudulent credit card transactions using statistical and machine learning models
#### March 8th, 2019

#### Authors: Angad S. Kalra, Shuang Di, Pulkit Mathur
#### Collaborators: Shobhit Jain and Adam Rahman

### Introduction
(TODO...)

###  Methods

#### Feature Engineering 
1. Transform Transaction_Type column into a one-hot encoding using Pandas get_dummies() function.
2. Remove these columns: 'Time_Stamp', 'Merchant_Id', 'Merchant_Old_Balance', 'Merchant_New_Balance', 'Is_Flagged_Fraud'. 
3. Create feature '%_of_balance' by performing transaction-wise division: Amount/Client_Old_Balance.  
4. Combine Client_Old_Balance and Client_New_Balance into one feature by taking the difference in values. Remove Client_Old_Balance and Client_New_Balance Columns. 

#### Data Preprocessing
1. Standardize all numerical columns by subtracting each value in a column from the mean and dividing by the standard deviation. 

#### Undersample Majority Class
1. TODO

#### Model Training 
* TODO

#### Model Evaluation 
* TODO

### Results
(TODO...)

### Discussion
(TODO...)

### Limitations
(TODO...)